# Data Processing for North Korea News (Normalization v4)

This notebook processes `summary_df` and `nk_cities.csv` to map event locations to administrative regions with refined normalization logic (excluding broad terms).

In [1]:
import pandas as pd
import numpy as np
import re

## 1. Load summary_df

In [2]:
# Load the summary dataframe
summary_df = pd.read_csv('data/summary_df_final.csv')
print(f"Total rows: {len(summary_df)}")
summary_df.head()

Total rows: 19141


,id,summary,keywords,event_title,event_date,event_person,event_org,event_loc,p_rice(won/kg),p_corn(won/kg),p_usd(won/usd),job_cost
0,spnews_101404,신의주온실종합농장 건설장에서 지대정리와 잔디심기가 마감단계에 이르고 있다. 군민건설...,"신의주, 온실농장, 지대정리, 잔디심기, 건설",신의주온실종합농장 건설 마감단계,2025-11-17,NaN,NaN,"평안북도, 신의주",NaN,NaN,NaN,0.000318
1,spnews_101403,북한이 나무의 사름률을 높이는 과학기술제품으로 천연생물활성제를 소개했다. 이 제품은...,"나무, 사름률, 천연생물활성제, 제품, 북한","북한, 나무 사름률 높이는 제품 소개",2025-11-17,NaN,NaN,NaN,NaN,NaN,NaN,0.000335
2,spnews_101402,북한 개성시에서 식물방역소와 돼지종축장이 새로 건설되었다. 개성시식물방역소 건설에 ...,"개성시, 식물방역소, 돼지종축장, 건설, 농업","개성시, 식물방역소와 돼지종축장 신축",2025-11-17,NaN,"개성시식물방역소, 시남새온실, 시유기질복합비료공장",개성시,NaN,NaN,NaN,0.000313
3,spnews_101394,"가을 추수가 마무리되면서 북한의 곡물 가격이 하락세를 보이고 있다. 평양, 혜산, ...","곡물가, 하락, 가을 추수, 물가, 북한",가을 추수로 인한 곡물가 하락,2025-11-15,NaN,NaN,"평양시, 혜산시, 원산시, 사리원시",19850.0,4500.0,34600.0,0.000379
4,spnews_101392,북한 각지에서 어머니날을 경축하며 김일성-김정일 동상에 꽃다발을 바쳤다. 공산주의어...,"어머니날, 경축, 동상, 공연, 여성근로자",어머니날 경축 행사,2025-11-16,NaN,NaN,"평안남도, 황해북도, 강원도, 양강도, 나선시, 개성시",NaN,NaN,NaN,0.000316


## 2. Sample 100 rows

In [3]:
# Randomly sample 100 rows
test_data = summary_df.sample(n=100, random_state=42).copy()
print(f"Test data shape: {test_data.shape}")

Test data shape: (100, 12)


## 3. Load nk_cities.csv and Build Maps

In [4]:
# Load nk_cities with euc-kr encoding
nk_cities = pd.read_csv('data/nk_cities.csv', encoding='euc-kr')

# Preprocessing helper
def get_search_keys(name):
    if pd.isna(name): return [], None
    # Handle parentheses: "나선시(라선시)" -> parts: ["나선시", "라선시"]
    parts = re.split(r'[()]', name)
    parts = [p.strip() for p in parts if p.strip()]
    
    canonical_name = parts[0] # The first part is the canonical name
    
    keys = []
    for p in parts:
        # Strip suffixes '도', '시', '군', '구역' for search key
        key = p
        if key.endswith('도'): key = key[:-1]
        elif key.endswith('시'): key = key[:-1]
        elif key.endswith('군'): key = key[:-1]
        elif key.endswith('구역'): key = key[:-1]
        keys.append(key)
    return keys, canonical_name

provinces_map = {} # search_key -> canonical_full_name
cities_map = {}    # search_key -> {'full': canonical_full_name, 'province': province_canonical_name}

for idx, row in nk_cities.iterrows():
    # Process Province
    p_keys, p_canon = get_search_keys(row['도'])
    for k in p_keys:
        provinces_map[k] = p_canon
        
    # Process City
    c_keys, c_canon = get_search_keys(row['시'])
    for k in c_keys:
        cities_map[k] = {
            'full': c_canon,
            'province': p_canon # This might be None or a string
        }

# Manual additions for abbreviations and broader terms
abbr_map = {
    '평남': '평안남도',
    '평북': '평안북도',
    '함남': '함경남도',
    '함북': '함경북도',
    '황남': '황해남도',
    '황북': '황해북도',
    '양강': '양강도',
    '자강': '자강도',
    '강원': '강원도',
    '평안도': '평안도', # Broader term
    '황해도': '황해도', # Broader term
    '함경도': '함경도',  # Broader term
    '평안': '평안도' # Example 7: "평안" -> "평안도" (Assuming broader term)
}

for abbr, full in abbr_map.items():
    provinces_map[abbr] = full

print(f"Mapped {len(provinces_map)} province keys and {len(cities_map)} city keys.")
# Debug check
print("Sample Cities Map keys:", list(cities_map.keys())[:5])

Mapped 20 province keys and 29 city keys.
Sample Cities Map keys: ['원산', '문천', '남포', '개성', '나선']


## 4. Map event_loc with Refined Normalization Logic

In [5]:
# Broad term definitions
BROAD_TERMS_MAP = {
    "평안도": ["평안남도", "평안북도"],
    "함경도": ["함경남도", "함경북도"],
    "황해도": ["황해남도", "황해북도"]
}

def map_location_normalized(loc_str):
    if pd.isna(loc_str):
        return None
    
    found_provinces = set()
    found_cities = [] # List of dicts
    
    # 1. Search for Provinces
    for key, full_name in provinces_map.items():
        if key in loc_str:
            found_provinces.add(full_name)
            
    # 2. Search for Cities
    for key, info in cities_map.items():
        if key in loc_str:
            match_info = info.copy()
            match_info['key'] = key
            found_cities.append(match_info)
            
    # 3. Consolidate and Remove Redundancy
    
    # 3a. Identify implied provinces from found cities
    implied_provinces = set()
    for c in found_cities:
        if pd.notna(c['province']):
            implied_provinces.add(c['province'])
            
    # 3b. Remove found provinces if they are implied by the cities
    temp_provinces = set()
    for p in found_provinces:
        if p not in implied_provinces:
            temp_provinces.add(p)
    
    # 3c. Remove Broad Terms if Specific Terms are present
    # Check against both remaining provinces AND implied provinces (since implied ones are also "present")
    all_present_specific_provinces = temp_provinces.union(implied_provinces)
    
    final_provinces = set()
    for p in temp_provinces:
        is_redundant_broad = False
        if p in BROAD_TERMS_MAP:
            # Check if any specific term for this broad term is present
            for specific in BROAD_TERMS_MAP[p]:
                if specific in all_present_specific_provinces:
                    is_redundant_broad = True
                    break
        
        if not is_redundant_broad:
            final_provinces.add(p)
            
    # 4. Format Output
    final_results = set()
    
    # Add Remaining Provinces
    for p in final_provinces:
        final_results.add(p)
        
    # Add Cities (Format: "Province City" or "City")
    for c in found_cities:
        full_city = c['full']
        province = c['province']
        
        if pd.notna(province):
            final_results.add(f"{province} {full_city}")
        else:
            final_results.add(full_city)
            
    if not final_results:
        return None
        
    return ', '.join(sorted(list(final_results)))

# Apply the mapping
test_data['mapped_admin_region'] = test_data['event_loc'].apply(map_location_normalized)

# Show results
test_data[['event_loc', 'mapped_admin_region']].head(20)

,event_loc,mapped_admin_region
1199,평양시,평양시
6880,"황해북도, 황해남도","황해남도, 황해북도"
3826,평양시,평양시
1114,NaN,None
3920,"평안도, 함남, 황해도, 함북, 양강도","양강도, 평안도, 함경남도, 함경북도, 황해도"
17753,평양시,평양시
2626,NaN,None
14971,"경기도, 개성시",개성시
14799,평양시,평양시
10138,평양시,평양시


## 5. Verification

In [6]:
# Test cases based on user requirements
test_cases = [
    ("강원도 원산시, 평양, 원산, 강원도", "강원도 원산시, 평양시"),
    ("원산", "강원도 원산시"),
    ("평안남도, 원산시", "평안남도, 강원도 원산시"),
    ("평양시, 원산시", "강원도 원산시, 평양시"), # Sorted order
    ("평양, 평남, 원산시", "강원도 원산시, 평안남도, 평양시"), # Sorted order
    ("평양시, 개성시", "개성시, 평양시"), # Sorted order
    ("평남, 평안북도, 평안", "평안남도, 평안북도"), # Sorted order
    ("평남, 순천", "평안남도 순천시"),
    ("함경남도, 순천", "함경남도, 평안남도 순천시")
]

print("Running Verification Tests...")
for inp, expected in test_cases:
    res = map_location_normalized(inp)
    print(f"Input: {inp} \n  - Output: {res} \n  - Expected: {expected} \n\n")

Running Verification Tests...
Input: 강원도 원산시, 평양, 원산, 강원도 
  - Output: 강원도 원산시, 평양시 
  - Expected: 강원도 원산시, 평양시 


Input: 원산 
  - Output: 강원도 원산시 
  - Expected: 강원도 원산시 


Input: 평안남도, 원산시 
  - Output: 강원도 원산시, 평안남도 
  - Expected: 평안남도, 강원도 원산시 


Input: 평양시, 원산시 
  - Output: 강원도 원산시, 평양시 
  - Expected: 강원도 원산시, 평양시 


Input: 평양, 평남, 원산시 
  - Output: 강원도 원산시, 평안남도, 평양시 
  - Expected: 강원도 원산시, 평안남도, 평양시 


Input: 평양시, 개성시 
  - Output: 개성시, 평양시 
  - Expected: 개성시, 평양시 


Input: 평남, 평안북도, 평안 
  - Output: 평안남도, 평안북도 
  - Expected: 평안남도, 평안북도 


Input: 평남, 순천 
  - Output: 평안남도 순천시 
  - Expected: 평안남도 순천시 


Input: 함경남도, 순천 
  - Output: 평안남도 순천시, 함경남도 
  - Expected: 함경남도, 평안남도 순천시 


